- Setup and Imports: Import necessary libraries (e.g., cv2, matplotlib, numpy, pickle).

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import os
from osgeo import gdal
import cv2

- We downloaded two data packages from the [copernicus](https://browser.dataspace.copernicus.eu/) for August and November. The data contained satellite images of the Kaniv reservoir, from which we selected files B02: Blue band, B03: Green band, B04: Red band.

```
2024_august
2024_november

```


- Converted the uploaded photos in jp2 format to png format.

In [ ]:
def jp2_to_png(fp_in,fp_out,prefix):
    bandList = [band for band in os.listdir(fp_in) if band.endswith('.jp2')]

    for band in bandList:
        jp2_path = os.path.join(fp_in, band)  # Full path for the input JP2 file
        in_image = gdal.Open(jp2_path)
        
        if in_image is None:
            print(f"Failed to open {jp2_path}")
            continue
        driver = gdal.GetDriverByName("GTiff")
        if driver is None:
            print("GTiff driver not available")
            continue
        
        fp_tif = os.path.join(fp_in, band[:-4] + '.tif')  # Output path for the GeoTIFF
        
        if os.path.exists(fp_tif):# Check if output file already exists
            print(f"File {fp_tif} already exists, skipping.")
            continue
        out_image = driver.CreateCopy(fp_tif, in_image, 0) # Create the output image as a GeoTIFF
       
        if out_image is None:
            print(f"Failed to create {fp_tif}")

        in_image = None
        out_image = None
        
    fp_in = prefix
    band_02 = rasterio.open(fp_in + "B02_60m.tif")
    band_03 = rasterio.open(fp_in + "B03_60m.tif")
    band_04 = rasterio.open(fp_in + "B04_60m.tif")

    red = band_04.read(1)
    green = band_03.read(1)
    blue = band_02.read(1)

    rgb_composite_raw= np.dstack((red, green, blue))

    def normalize(band):
        band_min, band_max = (band.min(), band.max())
        return ((band-band_min)/((band_max - band_min)))

    red_n = normalize(red)
    green_n = normalize(green)
    blue_n = normalize(blue)

    rgb_composite_n= np.dstack((red_n, green_n, blue_n))

    def brighten(band):
        alpha=0.13
        beta=0
        return np.clip(alpha*band+beta, 0,255)

    red_b=brighten(red)
    blue_b=brighten(blue)
    green_b=brighten(green)

    red_bn = normalize(red_b)
    green_bn = normalize(green_b)
    blue_bn = normalize(blue_b)

    rgb_composite_bn= np.dstack((red_bn, green_bn, blue_bn))


    rgb_plot=plt.imshow(rgb_composite_bn, interpolation='lanczos')
    plt.axis('off')
    image_path = os.path.join(fp_out,'november.png')
    plt.savefig(image_path, dpi=200, bbox_inches='tight')
    plt.close('all')
    

prefix = "2024_november\T36UUA_20241107T090059_"
fp_out = "2024_november_png"
fp_in = "2024_november"
img_1 = jp2_to_png(fp_in,fp_out,prefix)

![Alt data](./2024_august_png/august.png)

![Alt data](./2024_november_png/november.png)

- Model Loading:

In [ ]:
# Load images
img1_path = "c:/Users/kybav/OneDrive/Desktop/Home_Work/Home_work_5/Intership_test/Intership_test/Task_2/2024_august_png/august.png"
img2_path = "c:/Users/kybav/OneDrive/Desktop/Home_Work/Home_work_5/Intership_test/Intership_test/Task_2/2024_november_png/november.png"

img1 = cv2.imread(img1_path, cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread(img2_path, cv2.IMREAD_GRAYSCALE)

# Verify that images loaded correctly
if img1 is None:
    print(f"Error: Could not load image at {img1_path}")
if img2 is None:
    print(f"Error: Could not load image at {img2_path}")

# Proceed only if images are loaded
if img1 is not None and img2 is not None:
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # Save the model (keypoints and descriptors)
    keypoints1 = [
        (kp.pt, kp.size, kp.angle, kp.response, kp.octave, kp.class_id) for kp in kp1
    ]
    keypoints2 = [
        (kp.pt, kp.size, kp.angle, kp.response, kp.octave, kp.class_id) for kp in kp2
    ]
    model_data = {
        "keypoints1": keypoints1,
        "descriptors1": des1,
        "keypoints2": keypoints2,
        "descriptors2": des2,
    }

    with open("sift_model.pkl", "wb") as f:
        pickle.dump(model_data, f)
    print("Model saved as sift_model.pkl")


- Feature Detection and Matching: Code to detect features, compute descriptors, and match keypoints between the images.

In [ ]:
img1_path = "2024_august_png/august.png"  # Load images
img2_path = "2024_november_png/november.png"

img1 = cv2.imread(img1_path)
img2 = cv2.imread(img2_path)

if img1 is None:
    print(f"Error loading {img1_path}")  # Check if images loaded correctly
if img2 is None:
    print(f"Error loading {img2_path}")

if img1 is not None and img2 is not None:
    with open("sift_model.pkl", "rb") as f:  # Load the model
        model_data = pickle.load(f)

    kp1 = [
        cv2.KeyPoint(
            x=pt[0][0],
            y=pt[0][1],
            _size=pt[1],
            _angle=pt[2],  # Reconstruct the keypoints
            _response=pt[3],
            _octave=pt[4],
            _class_id=pt[5],
        )
        for pt in model_data["keypoints1"]
    ]
    kp2 = [
        cv2.KeyPoint(
            x=pt[0][0],
            y=pt[0][1],
            _size=pt[1],
            _angle=pt[2],
            _response=pt[3],
            _octave=pt[4],
            _class_id=pt[5],
        )
        for pt in model_data["keypoints2"]
    ]

    des1 = model_data["descriptors1"]  # Load descriptors
    des2 = model_data["descriptors2"]

    bf = cv2.BFMatcher()  # Matching descriptors
    matches = bf.knnMatch(des1, des2, k=2)

    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good.append([m])
    # Draw matches on the images
    img3 = cv2.drawMatchesKnn(
        img1,
        kp1,
        img2,
        kp2,
        good,
        None,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
    )

    output_path = "output_matches.png"  # Save the output image with matches to the specified directory
    cv2.imwrite(output_path, img3)  # Save the image in BGR format (OpenCV's default)

    plt.figure(
        figsize=(12, 6)
    )  # Optionally, display the image using Matplotlib, converting BGR to RGB for correct display
    plt.imshow(
        cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)
    )  # Convert BGR to RGB for correct colors
    plt.axis("off")  # Hide axis
    plt.show()

    print(f"Image saved to {output_path}")


![Alt data](./output_matches.png)

In your latest image with matches displayed, you can see two photos (for example, one from August and one from November) with keypoints highlighted and connected by lines. These lines indicate matches (correspondences) between objects in both images, identified through the SIFT (Scale-Invariant Feature Transform) algorithm or another feature detection method.

Meaning of the lines: The lines connect points that have a similar structure or texture in both images, showing that the algorithm found these areas to be similar.

Importance of these matches: Matches are useful for tasks like comparing changes in terrain, aligning images for further analysis, or detecting seasonal differences (fall vs. summer). They’re also used in 3D model construction, merging seasonal images, or even geodetic studies.